### Modelling the problem

Modelling a small example where we have $X$ employees, and $Y$ days.

Let $x \in X$ and $y \in Y$.

$\forall x \in X$ and $\forall y \in Y$

If employee $x$ has day $y$ off, $L (x,y) = 1$

Therefore $L$ is an $X$ by $Y$ matrix.

$$
L_{x,y} = 
\begin{pmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,y} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,y} \\
\vdots & \vdots & \ddots & \vdots \\
a_{x,1} & a_{x,2} & \cdots & a_{x,y} 
\end{pmatrix}
$$

Let us limit this to only a week for testing purposes, and only use a small number of staff members.

Let $X = 10$ and $Y = 5$.

We also have an employee quota ( $q$ ) of $75\%$ meaning that this percentage of employees must be present at all times. This is a hard constraint.

Therefore, 
$$\forall y \in Y,  \sum_{x=0}^X L(x,y) \geq (|X| * .75)$$

Finally, there is a staff leave allowance ( $a$ ) of maximum 2 days per person. This is a hard constraint. In a real model, this would be equal to the staff member's remaining holiday entitlement.

Therefore, 
$$\forall x \in X, \sum_{y=0}^Y L(x,y) \leq 2  $$

The preference of leave assignments for all employees in range $X$ is given by an $X * Y$ matrix $P$, where if $P(x,y) = 1$, the employee has requested this day off.

If an employee does not request a day off, then we do not want the algorithm to assign a day off to them.

Therefore,

$$ L(x,y) \leq P(x,y)  \quad \forall x \in X , \forall y \in Y $$



In [1]:
from ortools.sat.python import cp_model
import numpy as np
import pandas as pd


In [2]:
num_staff = 10
num_days = 5
staff_limit = [.75] * num_days


num_ones = int(num_staff * num_days * 0.9)
num_zeros = (num_staff * num_days) - num_ones
array = np.array([1] * num_ones + [0] * num_zeros)
np.random.shuffle(array)

p = array.reshape(num_staff,num_days)


print(f"Preference Matrix: \n {p}")



Preference Matrix: 
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [0 1 1 1 1]
 [1 1 1 1 1]
 [1 0 1 1 1]
 [0 1 1 1 1]
 [1 1 1 1 1]
 [1 1 0 1 0]]


In [3]:
model = cp_model.CpModel()

If employee $x$ has day $y$ off, $L (x,y) = 1$

In [4]:
l = {}
for x in range(num_staff):
    for y in range(num_days):
        l[(x, y)] = model.new_bool_var(f"L_{x}_y{y}")

Ensuring holiday entitlement is not exceeded (2 day limit).

$\forall x \in X, \sum_{y=0}^Y L(x,y) \leq 2  $


In [5]:
for x in range(num_staff):
    model.Add(sum(l[x, y] for y in range(num_days)) <= 2)

Ensuring that minimum staff coverage is satisfied (75% minimum)

$\forall y \in Y,  \sum_{x=0}^X L(x,y) \geq (|X| * .75)$

In [6]:
min_staff_required = int(np.ceil(0.75 * num_staff))


for y in range(num_days):
    model.Add(sum(l[x, y] for x in range(num_staff)) <= num_staff - min_staff_required)

    # amount of staff on leave each day doesn't exceed 25% of total staff


Objective function - maximise the leave which matches a persons preference, and minimise leave that doesn't match a preference.

The line below aims to maximise $P( x , y ) \times L( x , y )$ for each staff member 
$ x \in X $ and for each day $ y \in Y $ .

If a staff member hasn't asked for a day off ( meaning $ P(x,y) = 0 $ ), the algorithm shouldn't give them a day off, as $ P( x , y ) \times L( x , y ) = 0 \times 1 = 0 $

Likewise, if a staff member has asked for a day off ( meaning $ P(x,y) = 1 $ ), the algorithm should try to give them a day off, as $ P( x , y ) \times L( x , y ) = 1 \times 1 = 1 $




In [7]:
model.Maximize(sum(p[x][y] * l[x, y] for x in range(num_staff) for y in range(num_days)))

In [8]:
# Solve model

solver = cp_model.CpSolver()
# solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model)

In [9]:

def highlight_cells(val):
    color = '#00ff15' if val == 1 else ''
    return 'color: %s' % color

# function taken from stack overflow
def highlight_diff(data, other, color='#ff616b'):
    attr = f'background-color: {color}'
    return pd.DataFrame(np.where(data.ne(other), attr, ''),
                        index=data.index, columns=data.columns)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

    solutionArray = [[solver.Value(l[s,d]) for d in range(num_days)] for s in range(num_staff)]

    df = pd.DataFrame(solutionArray, columns=[f'Day {d+1}' for d in range(num_days)], index=[f'Employee {x+1}' for x in range(num_staff)])

    df2 = pd.DataFrame(p, columns=[f'Day {d+1}' for d in range(num_days)], index=[f'Employee {x+1}' for x in range(num_staff)])


    print("Solution\nRed cells are cells where preference matrix and solution matrix differ.")
    df_styled = df.style.apply(highlight_diff, axis=None, other=df2).applymap(highlight_cells)
    display(df_styled)

    print("Preference Matrix")
    df2_styled = df2.style.applymap(highlight_cells)
    display(df2_styled)

else:
    print("No feasible solution found.")

Solution
Red cells are cells where preference matrix and solution matrix differ.


C:\Users\seren\AppData\Local\Temp\ipykernel_25004\921478126.py:21: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_styled = df.style.apply(highlight_diff, axis=None, other=df2).applymap(highlight_cells)


,Day 1,Day 2,Day 3,Day 4,Day 5
Employee 1,0,1,1,0,0
Employee 2,0,0,0,1,1
Employee 3,0,0,1,1,0
Employee 4,0,1,0,0,1
Employee 5,0,0,0,0,0
Employee 6,1,0,0,0,0
Employee 7,0,0,0,0,0
Employee 8,0,0,0,0,0
Employee 9,1,0,0,0,0
Employee 10,0,0,0,0,0


Preference Matrix


C:\Users\seren\AppData\Local\Temp\ipykernel_25004\921478126.py:25: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df2_styled = df2.style.applymap(highlight_cells)


,Day 1,Day 2,Day 3,Day 4,Day 5
Employee 1,1,1,1,1,1
Employee 2,1,1,1,1,1
Employee 3,1,1,1,1,1
Employee 4,1,1,1,1,1
Employee 5,0,1,1,1,1
Employee 6,1,1,1,1,1
Employee 7,1,0,1,1,1
Employee 8,0,1,1,1,1
Employee 9,1,1,1,1,1
Employee 10,1,1,0,1,0


In [10]:

class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, num_staff, num_days, l, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._num_staff = num_staff
        self._num_days = num_days
        self._leave = l
        self._solution_count = 0
        self._solution_limit = limit


    def on_solution_callback(self):

        self._solution_count += 1

        print(f"Solution {self._solution_count}")

        for d in range(self._num_days):
            print(f"Day {d+1}")
            for s in range(self._num_staff):
                is_working = False
                if self.value(self._leave[(s, d)]):
                    is_working = True
                    print(f"  Employee {s+1} granted leave")
                if not is_working:
                    print(f"  Employee {s+1} does not work")

        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()



    def solutionCount(self):
        return self._solution_count



In [12]:
solution_limit = 5
solution_printer = SolutionPrinter(num_staff, num_days, l, solution_limit)

solver = cp_model.CpSolver()
# solver.parameters.linearization_level = 0
solver.parameters.log_search_progress = True
solver.parameters.enumerate_all_solutions = True
solver.log_callback = print  # (str)->None
solver.parameters.log_to_stdout = False

solver.Solve(model, solution_printer)

# Statistics.
print("\nStatistics")
print(f"  - conflicts: {solver.num_conflicts}")
print(f"  - branches : {solver.num_branches}")
print(f"  - wall time: {solver.wall_time}s")


Starting CP-SAT solver v9.11.4210
Parameters: log_search_progress: true enumerate_all_solutions: true log_to_stdout: false

Initial optimization model '': (model_fingerprint: 0xfebb1402d8760de5)
#Variables: 50 (#bools: 45 in objective)
  - 50 Booleans in [0,1]
#kLinearN: 15 (#terms: 100)

Starting presolve at 0.00s
  3.31e-04s  0.00e+00d  [operations_research::sat::CpModelPresolver::PresolveToFixPoint] #num_loops=1 
  1.26e-05s  0.00e+00d  [operations_research::sat::CpModelPresolver::ExtractEncodingFromLinear] 
  3.15e-05s  0.00e+00d  [operations_research::sat::CpModelPresolver::DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.57e-04s  1.15e-05d  [operations_research::sat::CpModelPresolver::Probe] #probed=100 
  4.00e-06s  0.00e+00d  [MaxClique] 
  5.84e-05s  0.00e+00d  [operations_research::sat::CpModelPresolver::PresolveToFixPoint] 
  4.10e-05s  0.00e+00d  [operations_research::sat::CpModelPresolver::ProcessAtMostOneAndLinear] 
  2.19e-05s  0.00e+00d  [operations_research::